In [1]:
!pip install torch transformers auto-gptq autoawq optimum onnx onnxruntime

  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 72.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 42.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline

In [4]:
clinc = load_dataset("clinc_oos", "plus")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
sample = clinc["train"][0]
print(sample)

# Каждый объект датасета содержит текст и соответствующее ему намерение.

{'text': 'what expression would i use to say i love you if i were an italian', 'intent': 61}


In [6]:
intents = clinc["train"].features["intent"]
intent = intents.int2str(sample["intent"])
print(intent)

translate


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = intents.num_classes

checkpoint = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(checkpoint, num_labels=num_labels).to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [9]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
print("Base Model: ", model.num_parameters())

Base Model:  1543714304


In [7]:
sample_input = clinc['train']['text'][101]

print(clinc['train']['text'][101])
print(clinc['train']['intent'][101])

complete a transaction from savings to checking of $20000
133


In [11]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
#WARMUP
for _ in range(10):
  _ = pipe(sample_input)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = pipe(sample_input)
total_time_model = time.time() - start
print("Общее время обработки 100 запросов базовой моделью:", total_time_model)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Общее время обработки 100 запросов базовой моделью: 83.49490714073181


## 1. Оптимизация через optimum

In [10]:
from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTOptimizer, ORTModelForCausalLM
from optimum.onnxruntime.configuration import OptimizationConfig

In [11]:
import onnxruntime

session_options = onnxruntime.SessionOptions()
session_options.log_severity_level = 0

In [ ]:
optimum_model1 = pipeline(
        task="text-generation",
        model=checkpoint,
        tokenizer=tokenizer,
        accelerator="ort",
        device=0
)

We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:460: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/usr/local/lib/python3.11/dist-packages/transformers/models/qwen2/modeling_qwen2.py:1067: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.shape[-1

Через optimum colab либо виснет, либо падает с ошибкой:  KeyError: 'qwen2 is not supported yet with the onnx backend. Only [] are supported. If you want to support onnx please propose a PR or open up an issue.'

In [ ]:
model_id = checkpoint
save_dir = "optimized_model"

optimum_model2 = ORTModelForCausalLM.from_pretrained(
    model_id,
    export=True,
    provider="CUDAExecutionProvider",
    session_options=session_options
)

/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:457: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  or not self.key_cache[layer_idx].numel()  # the layer has no cache
/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:285: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif sliding_window is None or key_value_length < sliding_window:
/usr/local/lib/python3.11/dist-packages/transformers/models/qwen2/modeling_qwen2.py:719: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be in

Через optimum colab либо виснет, либо падает с ошибкой:  KeyError: 'qwen2 is not supported yet with the onnx backend. Only [] are supported. If you want to support onnx please propose a PR or open up an issue.'

## 2. Оптимизация напрямую

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
model.eval()

input_text = "How are you going to spend holidays?"
inputs = tokenizer(input_text, return_tensors="pt")

if hasattr(model, "config") and hasattr(model.config, "use_cache"):
    model.config.use_cache = False  # отключаем кэширование

torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    "res/qwen2_1.5b_instruct.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size", 1: "sequence_length"},
    },
    opset_version=15,
    do_constant_folding=False,
)

print("Экспорт в ONNX завершен")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:285: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif sliding_window is

Экспорт в ONNX завершен


In [4]:
import onnxruntime as ort

# Создание сессии ONNX Runtime
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"]  # Попробовать GPU, затем CPU
sess = ort.InferenceSession('/content/res/qwen2_1.5b_instruct.onnx', providers=providers)

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [10]:
def generate_text(prompt, sess, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]

    for _ in range(max_length):
        outputs = sess.run(None,
                           {"input_ids": input_ids,
        "attention_mask": inputs["attention_mask"]})
        next_token = outputs["logits"][:, -1:].argmax(dim=-1)
        input_ids = torch.cat([input_ids, next_token], dim=-1)

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

print(generate_text("ONNX — это", sess))

ValueError: Required inputs (['onnx::Gather_2']) are missing from input feed (['input_ids', 'attention_mask']).